In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DataType
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
%fs
ls "mnt/tokyoolymic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolymic/raw-data/,raw-data/,0,1771155860000
dbfs:/mnt/tokyoolymic/transformed-data/,transformed-data/,0,1771935547000


In [0]:
spark

In [0]:
athletes = spark.read.format('csv').option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolymic/raw-data/athletes.csv")
coaches = spark.read.format('csv').option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolymic/raw-data/coaches.csv")
entriesgender = spark.read.format('csv').option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolymic/raw-data/entriesgender.csv")
medals = spark.read.format('csv').option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolymic/raw-data/medals.csv")
teams = spark.read.format('csv').option("header", "true").option("inferSchema", "true").load("/mnt/tokyoolymic/raw-data/teams.csv")

In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesgender.printSchema()
medals.printSchema()
teams.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
# Top countries with Gold medals

top_gold_medals_by_country = medals.orderBy("Gold", ascending=False).select("TeamCountry","Gold").limit(10).show()

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
+--------------------+----+



In [0]:
# Calculate the average number of entries by gender  for each  discipline .
entriesgender = entriesgender.groupBy("Discipline").avg("Female","Male").show()

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-8696179263108897>, line 2
      1 # Calculate the average number of entries by gender  for each  discipline .
----> 2 entriesgender = entriesgender.groupBy("Discipline").avg("Female","Male").show()

AttributeError: 'NoneType' object has no attribute 'groupBy'

In [0]:
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-8696179263108899>, line 1
----> 1 average_entries_by_gender = entriesgender.withColumn(
      2     'Avg_Female', entriesgender['Female'] / entriesgender['Total']
      3 ).withColumn(
      4     'Avg_Male', entriesgender['Male'] / entriesgender['Total']
      5 )
      6 average_entries_by_gender.show()

AttributeError: 'NoneType' object has no attribute 'withColumn'

In [0]:
entriesgender.groupBy("Discipline") \
    .avg("Female", "Male") \
    .withColumnRenamed("avg(Female)", "Average Female") \
    .withColumnRenamed("avg(Male)", "Average Male") \
    .show()

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-8696179263108898>, line 1
----> 1 entriesgender.groupBy("Discipline") \
      2     .avg("Female", "Male") \
      3     .withColumnRenamed("avg(Female)", "Average Female") \
      4     .withColumnRenamed("avg(Male)", "Average Male") \
      5     .show()

AttributeError: 'NoneType' object has no attribute 'groupBy'

In [0]:
entriesgender.printSchema()
medals.printSchema()
teams.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
# Rename columns in entriesgender
entriesgender_renamed = entriesgender.select(
    col("Discipline"),
    col("Female"),
    col("Male"),
    col("Total").alias("TotalParticipants")
)

# Rename columns in medals
medals_renamed = medals.select(
    col("TeamCountry"),
    col("Gold"),
    col("Silver"),
    col("Bronze"),
    col("Total").alias("TotalMedals")
)

In [0]:
fact_athlete = athletes.join(
    entriesgender_renamed, "Discipline", "left"
).join(
    medals_renamed, athletes.Country == medals_renamed.TeamCountry, "left"
)

In [0]:
fact_athlete = fact_athlete.select(
    col("PersonName"),
    athletes.Country,
    col("Discipline"),
    col("Female"),
    col("Male"),
    col("TotalParticipants"),
    col("Gold"),
    col("Silver"),
    col("Bronze")
)

fact_athlete.show(5)

+-----------------+-------+-------------------+------+----+-----------------+----+------+------+
|       PersonName|Country|         Discipline|Female|Male|TotalParticipants|Gold|Silver|Bronze|
+-----------------+-------+-------------------+------+----+-----------------+----+------+------+
|  AALERUD Katrine| Norway|       Cycling Road|    70| 131|              201|   4|     2|     2|
|      ABAD Nestor|  Spain|Artistic Gymnastics|    98|  98|              196|   3|     8|     6|
|ABAGNALE Giovanni|  Italy|             Rowing|   257| 265|              522|  10|    10|    20|
|   ABALDE Alberto|  Spain|         Basketball|   144| 144|              288|   3|     8|     6|
|    ABALDE Tamara|  Spain|         Basketball|   144| 144|              288|   3|     8|     6|
+-----------------+-------+-------------------+------+----+-----------------+----+------+------+
only showing top 5 rows


In [0]:
dim_athletes = athletes.select(
    col("PersonName").alias("athlete_name"),
    col("Country").alias("country"),
    col("Discipline").alias("discipline")
).dropDuplicates()

dim_athletes.show()
dim_athletes.printSchema()

+--------------------+--------------------+-------------------+
|        athlete_name|             country|         discipline|
+--------------------+--------------------+-------------------+
|        ABBINGH Lois|         Netherlands|           Handball|
|       ABBOTT Monica|United States of ...|  Baseball/Softball|
|     ABDELAZIZ Farah|               Egypt|       Table Tennis|
| ABDALRASOOL Mohamed|               Sudan|               Judo|
|       ABASS Abobakr|               Sudan|           Swimming|
|   ABDELAZIZ Abdalla|               Egypt|             Karate|
|           ABALO Luc|              France|           Handball|
|      ABDALLA Maryam|               Egypt|  Artistic Swimming|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|     AALERUD Katrine|              Norway|       Cycling Road|
|   ABDEL LATIF Radwa|               Egypt|           Shooting|
|      ABDALLAH Shahd|               Egypt|  Artistic Swimming|
|      ABALDE Alberto|               Spa

In [0]:
dim_coaches = coaches.select(
    col("Name").alias("coach_name"),
    col("Country").alias("country"),
    col("Discipline").alias("discipline"),
    col("Event").alias("event")
).dropDuplicates()

dim_coaches.show()
dim_coaches.printSchema()

+--------------------+--------------------+-----------------+--------+
|          coach_name|             country|       discipline|   event|
+--------------------+--------------------+-----------------+--------+
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|    AMO AGUADO Pablo|               Spain|         Football|    NULL|
|       ALSHEHRI Saad|        Saudi Arabia|         Football|     Men|
|  BABER Gareth Colin|                Fiji|     Rugby Sevens|     Men|
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|           ALY Kamal|               Egypt|         Football|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|     

In [0]:
dim_teams = teams.select(
    col("TeamName").alias("team_name"),
    col("Country").alias("country"),
    col("Discipline").alias("discipline"),
    col("Event").alias("event")
).dropDuplicates()

dim_teams.show()
dim_teams.printSchema()

+-------------+--------------------+--------------+------------+
|    team_name|             country|    discipline|       event|
+-------------+--------------------+--------------+------------+
|          ROC|                 ROC|3x3 Basketball|       Women|
|       Poland|              Poland|3x3 Basketball|         Men|
|      Romania|             Romania|3x3 Basketball|       Women|
|        China|People's Republic...|3x3 Basketball|       Women|
|       Latvia|              Latvia|3x3 Basketball|         Men|
|        Japan|               Japan|3x3 Basketball|         Men|
|United States|United States of ...|3x3 Basketball|       Women|
|    Australia|           Australia|       Archery|  Mixed Team|
|   Bangladesh|          Bangladesh|       Archery|  Mixed Team|
|        China|People's Republic...|3x3 Basketball|         Men|
|        Italy|               Italy|3x3 Basketball|       Women|
|  Netherlands|         Netherlands|3x3 Basketball|         Men|
|          ROC|          

In [0]:
dim_disciplines = athletes.select("Discipline").union(coaches.select("Discipline")).dropDuplicates()
dim_disciplines = dim_disciplines.withColumnRenamed("Discipline", "discipline")

dim_disciplines.show()
dim_disciplines.printSchema()

+-------------------+
|         discipline|
+-------------------+
|             Boxing|
|             Rowing|
|  Baseball/Softball|
|               Judo|
|           Swimming|
|         Basketball|
|           Handball|
|Rhythmic Gymnastics|
|             Karate|
|          Badminton|
|          Athletics|
|           Shooting|
|Artistic Gymnastics|
|       Cycling Road|
|          Wrestling|
|       Table Tennis|
|          Taekwondo|
|      Weightlifting|
|  Artistic Swimming|
|            Fencing|
+-------------------+
only showing top 20 rows
root
 |-- discipline: string (nullable = true)



In [0]:
# -------------------------------
# Write Dimension Tables
# -------------------------------

dim_athletes.write.mode("overwrite") \
    .option("header", "true") \
    .csv("/mnt/tokyoolymic/transformed-data/dim_athletes")

dim_coaches.write.mode("overwrite") \
    .option("header", "true") \
    .csv("/mnt/tokyoolymic/transformed-data/dim_coaches")

dim_teams.write.mode("overwrite") \
    .option("header", "true") \
    .csv("/mnt/tokyoolymic/transformed-data/dim_teams")

dim_disciplines.write.mode("overwrite") \
    .option("header", "true") \
    .csv("/mnt/tokyoolymic/transformed-data/dim_disciplines")

In [0]:
# -------------------------------
# Write Fact Table
# -------------------------------

fact_athlete.write.mode("overwrite") \
    .option("header", "true") \
    .csv("/mnt/tokyoolymic/transformed-data/fact_athletes")

In [0]:
# Gold Table: Medals Summary by Country
gold_medals_by_country = medals.groupBy("TeamCountry") \
    .agg(
        F.sum("Gold").alias("Total_Gold"),
        F.sum("Silver").alias("Total_Silver"),
        F.sum("Bronze").alias("Total_Bronze"),
        F.sum("Total").alias("Total_Medals")
    ) \
    .orderBy(F.desc("Total_Gold"))

# Save
gold_medals_by_country.write.mode("overwrite") \
    .option("header","true") \
    .csv("/mnt/tokyoolymic/gold-data/gold_medals_by_country")

In [0]:
# Gold Table: Gender Participation Summary

gold_gender_avg = entriesgender.withColumn(
    "Average_Female", F.col("Female") / F.col("Total")
).withColumn(
    "Average_Male", F.col("Male") / F.col("Total")
).select("Discipline","Average_Female","Average_Male")

gold_gender_avg.write.mode("overwrite") \
    .option("header","true") \
    .csv("/mnt/tokyoolymic/gold-data/gold_gender_avg")

In [0]:
# Gold Table: Athlete Summary

# Join athletes with medals by country (example)
athlete_summary = athletes.join(
    medals.groupBy("TeamCountry").agg(F.sum("Gold").alias("Gold_Total")), 
    athletes.Country == medals.TeamCountry, 
    "left"
).select(
    "PersonName","Country","Discipline","Gold_Total"
)

athlete_summary.write.mode("overwrite") \
    .option("header","true") \
    .csv("/mnt/tokyoolymic/gold-data/gold_athlete_summary")

In [0]:
# Gold Table: Coach Summary

coach_summary = coaches.groupBy("Country","Discipline") \
    .agg(F.count("Name").alias("Number_of_Coaches"))

coach_summary.write.mode("overwrite") \
    .option("header","true") \
    .csv("/mnt/tokyoolymic/gold-data/gold_coach_summary")